In [1]:
import numpy as np 
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from keras.layers import Dropout, Input, Concatenate,concatenate
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras import optimizers
from keras.callbacks import ModelCheckpoint
import tensorflow as tf

import keras.backend as K

Using TensorFlow backend.


In [2]:
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

In [3]:
from utils import *
df = functions.parse_file(r"raw_data/EmoContext/train2.txt", "EmoContext")
df.head(5)

,id,turn1,turn2,turn3,label
0,0,Don't worry I'm girl,hmm how do I know if you are,What's ur name?,others
1,1,When did I?,saw many times i think -_-,No. I never saw you,angry
2,2,By,by Google Chrome,Where you live,others
3,3,U r ridiculous,I might be ridiculous but I am telling the truth.,U little disgusting whore,angry
4,4,Just for time pass,wt do u do 4 a living then,Maybe,others


In [4]:
text_data = []
for idx,row in df.iterrows():
    text_data.append("{}. {}. {}.".format(row['turn1'], row['turn2'], row['turn3']))

In [5]:
NR_WORDS = 5000
MAX_PROP_LENGTH = 128
tokenizer = Tokenizer(num_words=NR_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                   lower=True,split=' ')

tokenizer.fit_on_texts(text_data)

X_train_1 = tokenizer.texts_to_sequences(df["turn1"])
X_train_1 = pad_sequences(X_train_1, maxlen = MAX_PROP_LENGTH)

X_train_2 = tokenizer.texts_to_sequences(df["turn2"])
X_train_2 = pad_sequences(X_train_2, maxlen = MAX_PROP_LENGTH)

X_train_3 = tokenizer.texts_to_sequences(df["turn3"])
X_train_3 = pad_sequences(X_train_3, maxlen = MAX_PROP_LENGTH)

In [6]:
def one_hot_vector(word,label=None):
    words = {"others": 0, "angry": 1, "sad":2, "happy": 3}
    if label == None:
        y = [0,0,0,0]
        y[words[word]] = 1
        return y
    if label == word:
        return [1,0]
    return [0,1]

Y_train = []

for idx,row in df.iterrows():
    Y_train.append(one_hot_vector(row['label']))

Y_train = np.array(Y_train)

In [7]:
df = functions.parse_file(r"raw_data/EmoContext/devwithlabels2.txt", "EmoContext")
df.head(5)

,id,turn1,turn2,turn3,label
0,0,Then dont ask me,YOURE A GUY NOT AS IF YOU WOULD UNDERSTAND,IM NOT A GUY FUCK OFF,angry
1,1,Mixed things such as??,the things you do.,Have you seen minions??,others
2,2,Today I'm very happy,and I'm happy for you ❤,I will be marry,happy
3,3,Woah bring me some,left it there oops,Brb,others
4,4,it is thooooo,I said soon master.,he is pressuring me,others


In [8]:
text_data = []
for idx,row in df.iterrows():
    text_data.append("{}. {}. {}.".format(row['turn1'], row['turn2'], row['turn3']))

In [9]:
tokenizer = Tokenizer(num_words=NR_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                   lower=True,split=' ')

tokenizer.fit_on_texts(text_data)

X_test_1 = tokenizer.texts_to_sequences(df["turn1"])
X_test_1 = pad_sequences(X_test_1, maxlen = MAX_PROP_LENGTH)

X_test_2 = tokenizer.texts_to_sequences(df["turn2"])
X_test_2 = pad_sequences(X_test_2, maxlen = MAX_PROP_LENGTH)

X_test_3 = tokenizer.texts_to_sequences(df["turn3"])
X_test_3 = pad_sequences(X_test_3, maxlen = MAX_PROP_LENGTH)

In [10]:
def one_hot_vector(word,label=None):
    words = {"others": 0, "angry": 1, "sad":2, "happy": 3}
    if label == None:
        y = [0,0,0,0]
        y[words[word]] = 1
        return y
    if label == word:
        return [1,0]
    return [0,1]

Y_test = []

for idx,row in df.iterrows():
    Y_test.append(one_hot_vector(row['label']))

Y_test = np.array(Y_test)

In [11]:
embed_dim = 128
lstm_out = 64
batch_size = 64

adam = optimizers.Adam(lr=0.01)
rmsprop = optimizers.RMSprop(lr=0.005, rho=0.9, epsilon=None, decay=0.0)

I_1 = Input(shape=X_train_1.shape[1:])
O_1 = Embedding(NR_WORDS, embed_dim,input_length = X_train_1.shape[1])(I_1)
O_1 = LSTM(lstm_out, dropout=0.5)(O_1)

I_2 = Input(shape=X_train_2.shape[1:])
O_2 = Embedding(NR_WORDS, embed_dim,input_length = X_train_2.shape[1])(I_2)
O_2 = LSTM(lstm_out, dropout=0.5)(O_2)

I_3 = Input(shape=X_train_3.shape[1:])
O_3 = Embedding(NR_WORDS, embed_dim,input_length = X_train_3.shape[1])(I_3)
O_3 = LSTM(lstm_out, dropout=0.5)(O_3)

result = concatenate([O_1,O_2,O_3])
result = Dense(128)(result)
result = Dropout(0.4)(result)
result = Dense(64)(result)
result = Dropout(0.4)(result)
result = Dense(4,activation='softmax')(result)
model = Model(inputs=[I_1,I_2,I_3], outputs=result,name="separate_embedings_model")
model.compile(loss = 'categorical_crossentropy', optimizer=rmsprop, metrics = ['accuracy', f1])

In [12]:
model.build(np.array([X_train_1,X_train_2,X_train_3]).shape)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 128)          0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 128)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 128, 128)     640000      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_

In [13]:
mdcheck = ModelCheckpoint("trained_models/best_model_val_acc{val_acc:.4f}.h5", monitor='val_f1', save_best_only=True)

In [14]:
Y_train = np.array(Y_train)
total = len(Y_train)
try:
    history = model.fit([X_train_1,X_train_2,X_train_3], Y_train,
                        validation_data=([X_test_1,X_test_2,X_test_3], Y_test),
                        epochs=20, verbose=1, batch_size=batch_size,
                        class_weight={
                            0: total / len(np.where(Y_train[:,0]==1.0)[0]),
                            1: total / len(np.where(Y_train[:,1]==1.0)[0]),
                            2: total / len(np.where(Y_train[:,2]==1.0)[0]),
                            3: total / len(np.where(Y_train[:,3]==1.0)[0]),
                        },callbacks=[mdcheck],shuffle=True)
except KeyboardInterrupt:
    pass

Train on 30160 samples, validate on 2755 samples
Epoch 1/20
30160/30160 [==============================] - 209s 7ms/step - loss: 2.4726 - acc: 0.7402 - f1: 0.6897 - val_loss: 1.4324 - val_acc: 0.5485 - val_f1: 0.2219
Epoch 2/20
30160/30160 [==============================] - 204s 7ms/step - loss: 1.4615 - acc: 0.8611 - f1: 0.8427 - val_loss: 1.4733 - val_acc: 0.5851 - val_f1: 0.2381
Epoch 3/20
30160/30160 [==============================] - 203s 7ms/step - loss: 1.1948 - acc: 0.8887 - f1: 0.8751 - val_loss: 1.5730 - val_acc: 0.5815 - val_f1: 0.2356
Epoch 4/20
15328/30160 [==============>...............] - ETA: 1:43 - loss: 0.9930 - acc: 0.9056 - f1: 0.8947

In [15]:
save_model(model)

In [16]:
df_test = functions.parse_file(r"raw_data/EmoContext/devwithlabels2.txt", "EmoContext")
df_test.head()

,id,turn1,turn2,turn3
0,0,Then dont ask me,YOURE A GUY NOT AS IF YOU WOULD UNDERSTAND,IM NOT A GUY FUCK OFF
1,1,Mixed things such as??,the things you do.,Have you seen minions??
2,2,Today I'm very happy,and I'm happy for you ❤,I will be marry
3,3,Woah bring me some,left it there oops,Brb
4,4,it is thooooo,I said soon master.,he is pressuring me


In [17]:
res = model.predict([X_test_1,X_test_2,X_test_3], batch_size=64, verbose=1)
res[3]

2755/2755 [==============================] - 4s 1ms/step


array([0.07798924, 0.03819224, 0.87913007, 0.00468843], dtype=float32)

In [18]:
revers_words = {0:"others", 1:"angry", 2:"sad", 3:"happy"}

def softmax_convert(res):
    max_i = 0
    max_v = 0
    for i in range(0,4):
        if res[i] > max_v:
            max_v = res[i]
            max_i = i
    return revers_words[max_i]

In [19]:
results = []
for r in res:
    results.append(softmax_convert(r))
    
df_test['label'] = results
df_test.head(50)
df_test.to_csv("lstm_normal_12_epochs_cv.txt",index=False , sep="\t")